# Spark Commands for the match algorithm

Step 1: We need to import our libraries


In [1]:
import pyspark
import requests
import datetime
import concurrent.futures
import glob
import pandas as pd
from pyspark.sql import SparkSession 
import json 

Step 2: We need to start a spark session

In [2]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()


Step 3: This is the main algorithm

In [48]:
def main(hashed_column, noise):
    noise = noise
    hashed_column = hashed_column
    start = datetime.datetime.now()

    request = requests.get(f'http://cluster-a:9200//take_data/{hashed_column}/{noise}')
    url_content = request.content
    with open("/opt/workspace/a_download.csv", 'wb') as csv_file:
        csv_file.write(url_content)


    request = requests.get(f'http://cluster-b:9300//take_data/{hashed_column}/{noise}')
    url_content = request.content
    with open("/opt/workspace/b_download.csv", 'wb') as csv_file:
        csv_file.write(url_content)


    download_time = datetime.datetime.now() - start

    df_1 = spark.read.csv(path="/opt/workspace/a_download.csv", sep=",", header=True)
    df_2 = spark.read.csv(path="/opt/workspace/b_download.csv", sep=",", header=True)

    df_1.createOrReplaceTempView("a_cluster_data")
    df_2.createOrReplaceTempView("b_cluster_data")

    start = datetime.datetime.now()


    matched_data = spark.sql("  SELECT a.id as a_id, a.name as a_name, a.surname as a_surname, b.id as b_id, b.name as b_name, b.surname as b_surname   \
                                FROM a_cluster_data as a \
                                FULL OUTER JOIN b_cluster_data  as b\
                              ")  
    
    matched_data.createOrReplaceTempView("not_matched_data")

                                 
    #matched_data.repartition(1).write.mode('overwrite').csv(f"/opt/workspace/joined_result", header=True)

    joined_time = datetime.datetime.now() - start
    
    matched_data.createOrReplaceTempView("matched_data")
    
    #The number of the true positives
    TP = spark.sql("  SELECT *\
                      FROM matched_data \
                      WHERE matched_data.a_id == matched_data.b_id and matched_data.a_name == matched_data.b_name and matched_data.a_surname == matched_data.b_surname")

    #The number of the false positives
    FP = spark.sql("  SELECT *\
                      FROM matched_data \
                      WHERE matched_data.a_id != matched_data.b_id and matched_data.a_name == matched_data.b_name and matched_data.a_surname == matched_data.b_surname")

    #The number of the false negatives
    FN = spark.sql("  SELECT *\
                      FROM matched_data \
                      WHERE matched_data.a_id == matched_data.b_id and matched_data.a_id != 'Fake Index' and matched_data.b_id != 'Fake Index' and matched_data.a_name != matched_data.b_name and matched_data.a_surname != matched_data.b_surname ")
    
    TP = TP.count()
    FP = FP.count()
    FN = FN.count()
    
    precision = TP / ( TP + FP ) 
    
    recall    = TP / ( TP + FN )
    
    return pd.DataFrame([[noise, download_time, joined_time , precision, recall , TP, FP , FN]], columns=['noise', 'download_time', 'joined_time', 'precision', 'recall' , 'TP', 'FP', 'FN'])
    

In [51]:
hashed = 'id'
result = pd.DataFrame(None, columns=['noise', 'download_time', 'joined_time', 'precision', 'recall', 'TP' , 'FP' , 'FN'])
for x in range (0,1000,10):
    result = pd.concat([result, main(hashed, x)], axis=0)

result.to_csv('final_data.csv', encoding='utf-8', header=True, index=False)
print(result)

   noise          download_time            joined_time  precision    recall  \
0      0 0 days 00:00:00.084614 0 days 00:00:00.009028   0.975309  0.765504   
0     10 0 days 00:00:00.072021 0 days 00:00:00.010834   0.975309  0.765504   
0     20 0 days 00:00:00.101917 0 days 00:00:00.009735   0.975309  0.765504   
0     30 0 days 00:00:00.096663 0 days 00:00:00.010662   0.975309  0.765504   
0     40 0 days 00:00:00.161308 0 days 00:00:00.010672   0.975309  0.765504   
..   ...                    ...                    ...        ...       ...   
0    950 0 days 00:00:00.763908 0 days 00:00:00.011306   0.975369  0.765957   
0    960 0 days 00:00:01.165393 0 days 00:00:00.006684   0.975369  0.765957   
0    970 0 days 00:00:01.851275 0 days 00:00:00.008781   0.975490  0.766859   
0    980 0 days 00:00:00.665921 0 days 00:00:00.038845   0.975430  0.766409   
0    990 0 days 00:00:00.616860 0 days 00:00:00.009468   0.973105  0.766859   

     TP  FP   FN  
0   395  10  121  
0   395  10  

In [2]:
requests.get(f"http://cluster-a:9200//get_initial_data")

<Response [200]>

In [3]:
requests.get(f"http://cluster-b:9300//get_initial_data")

<Response [200]>

In [3]:
result = requests.get(f"http://cluster-b:9300//return_statistics")
print(result.content)
b_cluster_result = json.loads(result.content)
print(b_cluster_result)

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"\n  "http://www.w3.org/TR/html4/loose.dtd">\n<html>\n  <head>\n    <title>pandas.io.sql.DatabaseError: Execution failed on sql &#x27; SELECT count(*) as false_positive                                FROM a_joined_data, b_joined_data                                WHERE a_joined_data.[id] == b_joined_data.[id] and ( a_joined_data.[surname] != b_joined_data.[surname] or a_joined_data.[name] != b_joined_data.[name] ) &#x27;: no such column: b_joined_data.id // Werkzeug Debugger</title>\n    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css"\n        type="text/css">\n    <!-- We need to make sure this has a favicon so that the debugger does\n         not accidentally trigger a request to /favicon.ico which might\n         change the application\'s state. -->\n    <link rel="shortcut icon"\n        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">\n    <script src="?__debugger__=yes&amp

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
sql_result.count()

405

In [ ]:
    #print(f"TP = {TP} \nFP = {FP} \nFN = {FN}")

#     URLS = ['http://cluster-a:9200//take_data/', 
#             'http://cluster-b:9300//take_data/']
    
#     def send_data(url, myfiles):
#         myfiles =  {'file': open(myfiles,'rb')}  
#         requests.post(f'{url}', files = myfiles)
    
#     myfiles = list()
#     directoryPath= '/opt/workspace/a_joined_result/'
#     for file_name in glob.glob(directoryPath+'*.csv'):
#         myfiles.append(file_name)
    
#     directoryPath= '/opt/workspace/b_joined_result/'
#     for file_name in glob.glob(directoryPath+'*.csv'):
#         myfiles.append(file_name)
        
#     with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#         task = {executor.submit(send_data, url, myfile): url for url,myfile in zip(URLS, myfiles)}

#         for future in concurrent.futures.as_completed(task):
#             url = task[future]
#             try:
#                 data = future.result()
#             except Exception as exc:
#                 print('%r generated an exception: %s' % (url, exc))
  

#     result = requests.get(f"http://cluster-b:9300//return_statistics")
#     b_cluster_result = json.loads(result.content)
    
 
#     result = requests.get(f"http://cluster-a:9200//return_statistics")
#     a_cluster_result = json.loads(result.content)
    